In [18]:
import pickle
import cobra

In [19]:
# Load the results
with open('results.pkl', 'rb') as f:
    experiment = pickle.load(f)

In [20]:
# Load the COBRA model
alt_model = cobra.io.read_sbml_model("../../../../GEM-repos/mit1002-model/model.xml")

'' is not a valid SBML 'SId'.


In [21]:
def get_readable_metabolite_name(metabolite):
    return f"{metabolite.name} ({metabolite.id})"

In [22]:
#metabolite_id = "cpd00007_c0" # (O2)
# metabolite_id = "cpd00011_c0" # (CO2)
metabolite_id = "cpd00002_c0" # (ATP)

In [23]:
# Check if the metabolite exists in the model
if metabolite_id in alt_model.metabolites:
    metabolite = alt_model.metabolites.get_by_id(metabolite_id)
else:
    raise ValueError(f"Metabolite {metabolite_id} not found in the model.")

In [24]:
# Find all reactions containing the metabolite
reactions_with_metabolite = [reaction for reaction in alt_model.reactions if metabolite in reaction.metabolites]

In [25]:
reactions_with_metabolite

[<Reaction rxn00247_c0 at 0x2b02ab100>,
 <Reaction rxn00225_c0 at 0x2b0303c70>,
 <Reaction rxn00148_c0 at 0x2b0303cd0>,
 <Reaction rxn01100_c0 at 0x2b0313e80>,
 <Reaction rxn01123_c0 at 0x2b03255e0>,
 <Reaction rxn00770_c0 at 0x2b03367c0>,
 <Reaction rxn00175_c0 at 0x2b03360a0>,
 <Reaction rxn00615_c0 at 0x2b0336e80>,
 <Reaction rxn00216_c0 at 0x2b033fb80>,
 <Reaction rxn00077_c0 at 0x2b033f7f0>,
 <Reaction rxn01275_c0 at 0x2b0309fd0>,
 <Reaction rxn08173_c0 at 0x2b03587c0>,
 <Reaction rxn00646_c0 at 0x2b0374b50>,
 <Reaction rxn01673_c0 at 0x2b0374d90>,
 <Reaction rxn00107_c0 at 0x2b0374d00>,
 <Reaction rxn09240_c0 at 0x2b0386d00>,
 <Reaction rxn00179_c0 at 0x2b038e520>,
 <Reaction rxn08137_c0 at 0x2b0374a90>,
 <Reaction rxn03181_c0 at 0x2b03bfd90>,
 <Reaction rxn08017_c0 at 0x2b03d6a00>,
 <Reaction rxn00361_c0 at 0x2b03e03a0>,
 <Reaction rxn04384_c0 at 0x2b03868e0>,
 <Reaction rxn00100_c0 at 0x2b03f0fd0>,
 <Reaction rxn00789_c0 at 0x2b03e83a0>,
 <Reaction rxn00097_c0 at 0x2b04096a0>,


In [26]:
# Get the fluxes for a specific cycle (the first, index 0) of the simulation
fluxes = experiment.fluxes_by_species[''].iloc[0]

In [27]:
# Print the nonzero flux reactions containing the metabolite, flux, and bounds
for reaction in reactions_with_metabolite:
    reaction_id = reaction.id
    flux = fluxes[reaction_id]
    
    # Filter out reactions with near-zero flux (using a threshold, e.g., 1e-6)
    if abs(flux) > 1e-6:
        lower_bound = reaction.lower_bound
        upper_bound = reaction.upper_bound
        readable_reaction_string = " + ".join([f"{coefficient} {get_readable_metabolite_name(met)}" for met, coefficient in reaction.metabolites.items()])
        print(f"{reaction_id} | {reaction.name} | {readable_reaction_string} : {flux} | Bounds: [{lower_bound}, {upper_bound}]")

rxn00225_c0 | ATP:acetate phosphotransferase_c0 | -1.0 ATP_c0 (cpd00002_c0) + -1.0 Acetate_c0 (cpd00029_c0) + 1.0 ADP_c0 (cpd00008_c0) + 1.0 Acetylphosphate_c0 (cpd00196_c0) : -10.169321731 | Bounds: [-1000.0, 1000.0]
rxn01100_c0 | ATP:3-phospho-D-glycerate 1-phosphotransferase_c0 | -1.0 ATP_c0 (cpd00002_c0) + -1.0 3-Phosphoglycerate_c0 (cpd00169_c0) + 1.0 ADP_c0 (cpd00008_c0) + 1.0 1,3-Bisphospho-D-glycerate_c0 (cpd00203_c0) : -15.954897784 | Bounds: [-1000.0, 1000.0]
rxn00770_c0 | ATP:D-ribose-5-phosphate diphosphotransferase_c0 | -1.0 ATP_c0 (cpd00002_c0) + -1.0 ribose-5-phosphate_c0 (cpd00101_c0) + 1.0 AMP_c0 (cpd00018_c0) + 1.0 H+_c0 (cpd00067_c0) + 1.0 PRPP_c0 (cpd00103_c0) : 1.2904062556 | Bounds: [-1000.0, 1000.0]
rxn00615_c0 | ATP:glycerol 3-phosphotransferase_c0 | -1.0 ATP_c0 (cpd00002_c0) + -1.0 Glycerol_c0 (cpd00100_c0) + 1.0 ADP_c0 (cpd00008_c0) + 1.0 H+_c0 (cpd00067_c0) + 1.0 Glycerol-3-phosphate_c0 (cpd00080_c0) : 0.055191535838 | Bounds: [0.0, 1000.0]
rxn00077_c0 | ATP:

In [28]:
# Print the nonzero flux reactions containing the metabolite, flux, and bounds
for reaction in reactions_with_metabolite:
    reaction_id = reaction.id
    flux = fluxes[reaction_id]
    
    # Filter out reactions with near-zero flux (using a threshold, e.g., 1e-6)
    if abs(flux) < 1e-6:
        continue
    
    # Get reactions where CO2 is being used (i.e. the it is a product and the 
    # flux is negative or it is a reactant and the flux is positive)
    if (metabolite in reaction.products and flux > 0) or (metabolite in reaction.reactants and flux < 0):
        lower_bound = reaction.lower_bound
        upper_bound = reaction.upper_bound
        readable_reaction_string = " + ".join([f"{coefficient} {get_readable_metabolite_name(met)}" for met, coefficient in reaction.metabolites.items()])
        print(f"{reaction_id} | {reaction.name} | {readable_reaction_string} : {flux} | Bounds: [{lower_bound}, {upper_bound}]")

rxn00225_c0 | ATP:acetate phosphotransferase_c0 | -1.0 ATP_c0 (cpd00002_c0) + -1.0 Acetate_c0 (cpd00029_c0) + 1.0 ADP_c0 (cpd00008_c0) + 1.0 Acetylphosphate_c0 (cpd00196_c0) : -10.169321731 | Bounds: [-1000.0, 1000.0]
rxn01100_c0 | ATP:3-phospho-D-glycerate 1-phosphotransferase_c0 | -1.0 ATP_c0 (cpd00002_c0) + -1.0 3-Phosphoglycerate_c0 (cpd00169_c0) + 1.0 ADP_c0 (cpd00008_c0) + 1.0 1,3-Bisphospho-D-glycerate_c0 (cpd00203_c0) : -15.954897784 | Bounds: [-1000.0, 1000.0]
rxn08173_c0 | F(1)-ATPase_c0 | -1.0 Phosphate_c0 (cpd00009_c0) + -4.0 H+_e0 (cpd00067_e0) + -1.0 ADP_c0 (cpd00008_c0) + 3.0 H+_c0 (cpd00067_c0) + 1.0 ATP_c0 (cpd00002_c0) + 1.0 H2O_c0 (cpd00001_c0) : 79.628140985 | Bounds: [-1000.0, 1000.0]
rxn09176_c0 | polyphosphate kinase_c0 | -1.0 ATP_c0 (cpd00002_c0) + -1.0 Phosphate_c0 (cpd00009_c0) + 1.0 ADP_c0 (cpd00008_c0) + 1.0 PPi_c0 (cpd00012_c0) : -7.7277987564 | Bounds: [-1000.0, 1000.0]
rxn01517_c0 | ATP:dUMP phosphotransferase_c0 | -1.0 H+_c0 (cpd00067_c0) + -1.0 dUMP_c0 

In [29]:
# Get all the reactions that can produce ATP
metabolite_id = "cpd00002_c0" # (ATP)
metabolite = alt_model.metabolites.get_by_id(metabolite_id)

for reaction in alt_model.reactions:
    if metabolite in reaction.products:
        if 'bigg.reaction' in reaction.annotation.keys():
            bigg_id = reaction.annotation['bigg.reaction']
        else:
            bigg_id = 'No BiGG ID'
        lower_bound = reaction.lower_bound
        upper_bound = reaction.upper_bound
        readable_reaction_string = " + ".join([f"{coefficient} {get_readable_metabolite_name(met)}" for met, coefficient in reaction.metabolites.items()])
        print(f"{reaction.id} | {bigg_id} | Bounds: [{lower_bound}, {upper_bound}] | {reaction.name} | {readable_reaction_string}")
    if metabolite in reaction.reactants and reaction.lower_bound < 0:
        if 'bigg.reaction' in reaction.annotation.keys():
            bigg_id = reaction.annotation['bigg.reaction']
        else:
            bigg_id = 'No BiGG ID'
        lower_bound = reaction.lower_bound
        upper_bound = reaction.upper_bound
        readable_reaction_string = " + ".join([f"{coefficient} {get_readable_metabolite_name(met)}" for met, coefficient in reaction.metabolites.items()])
        print(f"{reaction.id} | {bigg_id} | Bounds: [{lower_bound}, {upper_bound}] | {reaction.name} | {readable_reaction_string}")

rxn00225_c0 | ACKr | Bounds: [-1000.0, 1000.0] | ATP:acetate phosphotransferase_c0 | -1.0 ATP_c0 (cpd00002_c0) + -1.0 Acetate_c0 (cpd00029_c0) + 1.0 ADP_c0 (cpd00008_c0) + 1.0 Acetylphosphate_c0 (cpd00196_c0)
rxn00148_c0 | ['CDC19', 'PYK', 'PYK2'] | Bounds: [-1000.0, 0.0] | ATP:pyruvate 2-O-phosphotransferase_c0 | -1.0 ATP_c0 (cpd00002_c0) + -1.0 Pyruvate_c0 (cpd00020_c0) + 1.0 ADP_c0 (cpd00008_c0) + 1.0 Phosphoenolpyruvate_c0 (cpd00061_c0) + 1.0 H+_c0 (cpd00067_c0)
rxn01100_c0 | ['PGK1', 'PGK'] | Bounds: [-1000.0, 1000.0] | ATP:3-phospho-D-glycerate 1-phosphotransferase_c0 | -1.0 ATP_c0 (cpd00002_c0) + -1.0 3-Phosphoglycerate_c0 (cpd00169_c0) + 1.0 ADP_c0 (cpd00008_c0) + 1.0 1,3-Bisphospho-D-glycerate_c0 (cpd00203_c0)
rxn00770_c0 | No BiGG ID | Bounds: [-1000.0, 1000.0] | ATP:D-ribose-5-phosphate diphosphotransferase_c0 | -1.0 ATP_c0 (cpd00002_c0) + -1.0 ribose-5-phosphate_c0 (cpd00101_c0) + 1.0 AMP_c0 (cpd00018_c0) + 1.0 H+_c0 (cpd00067_c0) + 1.0 PRPP_c0 (cpd00103_c0)
rxn08173_c0 | [